<a href="https://colab.research.google.com/github/HyunLee103/AI_Hackathon_konkuk/blob/gyu/tokenization-gyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizing 관련 노트북


## Install requirements - Only in colab

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 79.3MB/s 
     |████████████████████████████████| 3.8MB 51.8MB/s 
     |████████████████████████████████| 92kB 13.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


## Preprocessing

### Load Data


In [3]:
base_dir = '/content/drive/My Drive/ku-ai'

In [4]:
import os
import re

def load_data(path):
    sentences = []
    labels = []

    with open(path, 'r', encoding='utf-8') as fp:
        lines = fp.readlines()
        for line in lines:
            _, sentence, label = line.strip().split('\t')
            cleaned_sentence = re.sub('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+', ' ', sentence).strip()
            if cleaned_sentence:
                sentences.append(cleaned_sentence) 
                labels.append(label)

        return sentences, labels

train_sentence, train_label = load_data(os.path.join(base_dir, 'ratings_train.txt'))
test_sentence, test_label = load_data(os.path.join(base_dir, 'ratings_test.txt'))

train_sentence[:10]

['아 더빙 진짜 짜증나네요 목소리',
 '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 솔직히 재미는 없다 평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 세부터 초등학교 학년생인 살용영화 ㅋㅋㅋ 별반개도 아까움',
 '원작의 긴장감을 제대로 살려내지못했다',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지 정말 발로해도 그것보단 낫겟다 납치 감금만반복반복 이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나']

### Tokenization (형태소분석 및 조사는 따로 저장 - Stop Words 로 쓸 생각)

In [5]:
from konlpy.tag import Okt

tokenizer = Okt()
tokenized_train_sentence = []
tokenized_test_sentence = []
stop_words = []

for sentence in train_sentence:
    morphs_pos = tokenizer.pos(sentence)
    tokens = []
    for mp in morphs_pos:
        if mp[1] == 'Josa':
            stop_words.append(mp[0])
        tokens.append(mp[0])
    tokenized_train_sentence.append(tokens)

for sentence in test_sentence:
    morphs_pos = tokenizer.pos(sentence)
    tokens = []
    for mp in morphs_pos:
        if mp[1] == 'Josa':
            stop_words.append(mp[0])
        tokens.append(mp[0])
    tokenized_test_sentence.append(tokens)

print(tokenized_train_sentence[:10])
print(tokenized_test_sentence[:10])
print(stop_words[:10])

[['아', '더빙', '진짜', '짜증나네요', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나'], ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'], ['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정'], ['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다'], ['막', '걸음', '마', '뗀', '세', '부터', '초등학교', '학년', '생인', '살용', '영화', 'ㅋㅋㅋ', '별', '반개', '도', '아까', '움'], ['원작', '의', '긴장감', '을', '제대로', '살려내지못', '했다'], ['별', '반개', '도', '아깝다', '욕', '나온다', '이응경', '길용우', '연', '기', '생활', '이', '몇', '년', '인지', '정말', '발', '로', '해도', '그것', '보단', '낫겟다', '납치', '감금', '만', '반복', '반복', '이', '드라마', '는', '가족', '도', '없다', '연기', '못', '하는', '사람', '만', '모', '엿', '네'], ['액션', '이', '없는데도', '재미', '있는', '몇', '안되는', '영화'], ['왜케', '평점', '이', '낮은건데', '꽤', '볼', '만', '한', '데', '헐리우드', '식', '화려함에만', '너무', '길들여져', '있나']]
[['굳', 'ㅋ'], ['뭐', '야', '이', '평점', '들', '은', '나쁘진', '않지만', '점', '짜', '리', '는', '더', '더욱', '아니잖아'], ['지루하지는', '않은데', '완전', '

### 임시로 File 에 **저장**

#### (sentence, tokens, label) 형식으로 csv 파일로 저장

In [11]:
import pandas as pd
import numpy as np

train_data = np.array([train_sentence, tokenized_train_sentence, train_label], dtype=object)
test_data = np.array([test_sentence, tokenized_test_sentence, test_label], dtype=object)


train_df = pd.DataFrame(train_data.T, columns=['sentence', 'tokenized_sentence', 'label'])
test_df = pd.DataFrame(test_data.T, columns=['sentence', 'tokenized_sentence', 'label'])

train_df.to_csv(os.path.join(base_dir, 'train_data.csv'), index=False)
test_df.to_csv(os.path.join(base_dir, 'test_data.csv'), index=False)

#### 조사만 모아서 텍스트 파일에 띄어쓰기로 구분하여 저장

In [10]:
stop_words = ' '.join(stop_words)
with open(os.path.join(base_dir, 'stop_words.txt'), 'w', encoding='utf-8') as fp:
    fp.write(stop_words)

#### 개수 세서 저장

In [16]:
from collections import Counter

counter = Counter(stop_words.split()).most_common()
print(counter)

[('이', 52203), ('의', 40582), ('에', 36468), ('가', 35611), ('을', 30937), ('도', 26808), ('는', 22560), ('를', 20991), ('은', 20863), ('으로', 10544), ('로', 9609), ('에서', 9327), ('만', 9117), ('과', 8182), ('한', 7798), ('인', 6965), ('다', 6728), ('이다', 6620), ('와', 5564), ('고', 5324), ('게', 5138), ('까지', 5121), ('하고', 4745), ('요', 3816), ('보다', 3689), ('나', 3550), ('지', 2936), ('라', 2736), ('인데', 2728), ('라고', 2453), ('에게', 2160), ('이고', 2154), ('엔', 2113), ('이나', 2111), ('서', 2095), ('랑', 2060), ('면', 2056), ('부터', 2035), ('네', 1891), ('라는', 1837), ('야', 1816), ('아', 1680), ('냐', 1634), ('지만', 1488), ('인가', 1470), ('인지', 1356), ('이네', 1351), ('에는', 1324), ('이랑', 1264), ('이라', 1257), ('처럼', 1202), ('밖에', 1187), ('이라는', 1151), ('한테', 1052), ('에도', 1014), ('이지', 940), ('이라고', 936), ('니', 885), ('이란', 862), ('이야', 828), ('마다', 810), ('이지만', 774), ('니까', 765), ('여', 722), ('든', 716), ('보단', 675), ('라도', 551), ('에요', 519), ('보다는', 519), ('죠', 519), ('이라도', 510), ('라니', 474), ('말고', 473), ('에선', 467), (

In [17]:
with open(os.path.join(base_dir, 'josa_count.txt'), 'w', encoding='utf-8') as fp:
    for word, count in counter:
        fp.write('{}\t{}\n'.format(word, count))